In [1]:
%reload_ext autoreload
%autoreload 2
import slap2_py as spy
import wisco_slap as wis
import wisco_slap.defs as DEFS
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import matplotlib as mpl
import h5py
import glob
import polars as pl
from pathlib import Path
import tifffile
import seaborn as sns
import electro_py as epy
import cv2

import h5py
import numpy as np
from pathlib import Path
from typing import Any, Union
import h5py

# Complete Analysis Pipeline (post ophys-slap2-analysis)

In [ ]:
subject = 'sargas'
exp = 'exp_3'
loc = 'loc_B'
acq = 'acq_1'

## 1. Update Experiment info

In [ ]:
wis.pipes.exp_info.update_exp_info_spreadsheet() #update the experiment info spreadsheet
wis.peri.sync.update_sync_info(subject, exp, redo=False) # update the sync info
wis.pipes.exp_info._update_dmd_info(subject, exp, loc, acq) # update the dmd info
wis.pipes.annotation_materials.full_annotation_pipeline(subject, exp, loc, acq) # run the full annotation pipeline

#Get the sync block for subsequent steps
sb = wis.peri.sync.get_acq_sync_block(subject, exp, loc, acq)

# Generate the video frame stack for whisker annotation
wis.pipes.videos._save_video_frame_stack(subject, exp, sb, n_frames=20, redo=False)

## 2. Perform Annotations

1. Label averaged images on the canvas, notably tracing each DMD to its soma, and associating a dend-ID. Also estimate DMD depth here and mark on the image. Use the reference anatomy stacks which should be collected after the experiment.
2. Update dmd_info.yaml with the DMD depths labelled in #1.
3. label the soma_roi locations on their respective tiff images in napari.
4. use the syanpse labeller to classify all synapses with their soma, dend-ID, type, etc.
5. label the whisker pixels (open napari window, then open whisker stack and label) Note: can simply launch ROI viewer, close the ROI stacks, and open the video stack.

## 3. Save Extracted Dataframes

In [ ]:
wis.pipes.traces.gen_and_save_all_activity_dfs(subject, exp, loc, acq, overwrite=False)
wis.pipes.videos._run_pupil_inference(subject, exp, redo=False)
wis.pipes.videos._save_full_exp_eye_dfs(subject, exp, redo=False)
wis.pipes.videos._save_whisking_frame_differences(subject, exp, sb, redo=False)

## 4. Complete the Sleepscoring Pipeline

In [ ]:
# generate all scoring data
wis.pipes.sleepscore.full_scoring_data_pipeline(subject, exp, loc, acq, overwrite=False)

Now, pop open the sleepscoring data and update sb_scoring_times.yaml to specify the periods with good/usable data that can be fed to the model.

In [ ]:
# autoscore the experiment
wis.pipes.score_mi._autoscore_mi(subject, exp, sb, overwrite=False)